In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
from sklearn.model_selection import StratifiedKFold

커널초기화 : 르쿤초기화 Dense(kernel_initializer = 'lecun_normal')  
활성화함수 : selu Dense( activation = 'selu')  
정규화 : 배치정규화  
규제 : 알파 드롭아웃(drop_rate=최적화)  
옵티마이저 : 모멘텀 최적화 optimizer = keras.optimizers.SGD(lr = 최적화, momentum = 0.9, nesterov = True)

In [31]:
test = pd.read_csv('pima-indians-diabetes_test.data', header = None)
train = pd.read_csv('pima-indians-diabetes_train.data', header = None)

In [32]:
col = ['pregnant', 'glucose', 'bloodpressure', 'skin', 'ilsulin', 'bmi' ,'diabetes_faimily','age', 'class']
test.columns = col
train.columns = col

In [33]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pregnant          100 non-null    int64  
 1   glucose           100 non-null    int64  
 2   bloodpressure     100 non-null    int64  
 3   skin              100 non-null    int64  
 4   ilsulin           100 non-null    int64  
 5   bmi               100 non-null    float64
 6   diabetes_faimily  100 non-null    float64
 7   age               100 non-null    int64  
 8   class             100 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 7.2 KB


In [34]:
X_train = train.iloc[:,:8]
y_train = train.iloc[:,8]

In [35]:
X_test = test.iloc[:,:8]
y_test = test.iloc[:,8]

In [36]:
X_train

,pregnant,glucose,bloodpressure,skin,ilsulin,bmi,diabetes_faimily,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
663,9,145,80,46,130,37.9,0.637,40
664,6,115,60,39,0,33.7,0.245,40
665,1,112,80,45,132,34.8,0.217,24
666,4,145,82,18,0,32.5,0.235,70


In [37]:
def build_model(num_layers, num_nod, drop_rate, lr):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = 8))
    model.add(keras.layers.BatchNormalization())
    for iterzz in range(0,num_layers):
        model.add(keras.layers.Dense(num_nod, activation = 'selu', kernel_initializer = 'lecun_normal'))
        model.add(keras.layers.AlphaDropout(rate = drop_rate))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation = 'sigmoid', kernel_initializer = 'lecun_normal'))
    optimizer = keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return(model)

In [38]:
def k_fold(X, y, num_layers, num_nod, drop_rate, lr, k):
    folds = StratifiedKFold(n_splits = k, shuffle = True, random_state = 201600177)
    acc = []
    for train_idx, val_idx in folds.split(X,y):
        X_cv_train = X.iloc[train_idx,]
        y_cv_train = y[train_idx]
        X_cv_valid = X.iloc[val_idx,]
        y_cv_valid = y[val_idx]
        model = None
        model = build_model(num_layers, num_nod, drop_rate, lr)
        callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20)
        model.fit(X_cv_train, y_cv_train, epochs = 100, callbacks = callback, batch_size = 128, verbose = 0)
        acc.append(model.evaluate(X_cv_valid, y_cv_valid, verbose = 0)[1])
    return(sum(acc)/len(acc))

In [39]:
def exapnd_grid(num_layers, num_nods, drop_rate, lr):
    return(np.array([(aa, ab, round(ac,5), round(ad,5)) for aa in num_layers for ab in num_nods for ac in drop_rate for ad in lr]))

In [40]:
grid = exapnd_grid(np.arange(1,4), np.arange(20,210,30), np.arange(0.05,0.2,0.05), [0.0001, 0.001, 0.01, 0.1])

In [41]:
def grid_search(X, y, gr, k):
    accl = []
    start = time.time()
    for i,g in enumerate(gr):
        accl.append(k_fold(X, y, int(g[0]), int(g[1]), g[2], g[3], 5))
        print(f'{i}/{len(gr)}')
    return([gr[accl.index(max(accl))], max(accl), time.time() - start])

In [42]:
result = grid_search(X_train, y_train, grid, 5)

0/336
1/336
2/336
3/336
4/336
5/336
6/336
7/336
8/336
9/336
10/336
11/336
12/336
13/336
14/336
15/336
16/336
17/336
18/336
19/336
20/336
21/336
22/336
23/336
24/336
25/336
26/336
27/336
28/336
29/336
30/336
31/336
32/336
33/336
34/336
35/336
36/336
37/336
38/336
39/336
40/336
41/336
42/336
43/336
44/336
45/336
46/336
47/336
48/336
49/336
50/336
51/336
52/336
53/336
54/336
55/336
56/336
57/336
58/336
59/336
60/336
61/336
62/336
63/336
64/336
65/336
66/336
67/336
68/336
69/336
70/336
71/336
72/336
73/336
74/336
75/336
76/336
77/336
78/336
79/336
80/336
81/336
82/336
83/336
84/336
85/336
86/336
87/336
88/336
89/336
90/336
91/336
92/336
93/336
94/336
95/336
96/336
97/336
98/336
99/336
100/336
101/336
102/336
103/336
104/336
105/336
106/336
107/336
108/336
109/336
110/336
111/336
112/336
113/336
114/336
115/336
116/336
117/336
118/336
119/336
120/336
121/336
122/336
123/336
124/336
125/336
126/336
127/336
128/336
129/336
130/336
131/336
132/336
133/336
134/336
135/336
136/336
137/336
138/33

In [45]:
print(len(grid))
print(result)

336
[array([1.0e+00, 1.1e+02, 2.0e-01, 1.0e-01]), 0.7754685282707214, 5332.0907571315765]


In [50]:
tuned_model = build_model(int(result[0][0]), int(result[0][1]), result[0][2], result[0][3])
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20)
tuned_model.fit(X_train, y_train, epochs = 100, callbacks = callback, batch_size = 128)

Epoch 1/100
6/6 [==============================] - 0s 4ms/step - loss: 0.8442 - accuracy: 0.4506
Epoch 2/100
6/6 [==============================] - 0s 5ms/step - loss: 0.8258 - accuracy: 0.4611
Epoch 3/100
6/6 [==============================] - 0s 5ms/step - loss: 0.7586 - accuracy: 0.5240
Epoch 4/100
6/6 [==============================] - 0s 5ms/step - loss: 0.7469 - accuracy: 0.5419
Epoch 5/100
6/6 [==============================] - 0s 4ms/step - loss: 0.6923 - accuracy: 0.5689
Epoch 6/100
6/6 [==============================] - 0s 5ms/step - loss: 0.6687 - accuracy: 0.6198
Epoch 7/100
6/6 [==============================] - 0s 5ms/step - loss: 0.6294 - accuracy: 0.6527
Epoch 8/100
6/6 [==============================] - 0s 5ms/step - loss: 0.6382 - accuracy: 0.6272
Epoch 9/100
6/6 [==============================] - 0s 5ms/step - loss: 0.6381 - accuracy: 0.6602
Epoch 10/100
6/6 [==============================] - 0s 5ms/step - loss: 0.6164 - accuracy: 0.6632
Epoch 11/100
6/6 [===========

In [51]:
tuned_model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.4908 - accuracy: 0.7800


[0.490849107503891, 0.7799999713897705]

In [52]:
keras.__version__

'2.5.0'

In [55]:
tf.version.VERSION

'2.5.0'

In [56]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]